# Webscraping para obtener la población de CABA

Para elaborar uno de los KPI propuestos para este proyecto es necesario contar con la población de la Coudad Autónoma de Buenos Aires. Para ello, se decidió utilizar la técnica de web scraping para extraer dicha información desde una página de Wikipedia. Esta técnica consiste en acceder a una página web, analizar su estructura HTML y extraer los datos relevantes de la página de una manera automatizada.

A continuación, se detallan los pasos seguidos.

## Importaciones necesarias

In [26]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import utils

## Extracción de la información

Se define la URL de la página de Wikipedia que contiene la tabla con la población de los últimos censos realizados.

In [27]:
url = 'https://es.wikipedia.org/wiki/Buenos_Aires'

Se envía una solicitud HTTP para obtener el contenido de la página.

In [28]:
response = requests.get(url)

Se crea un objeto BeautifulSoup para analizar el contenido HTML de la página.

In [29]:
soup = BeautifulSoup(response.text, 'html.parser')

Se inspeccionó la página donde se encuentra la información para conocer la estructura de etiquetas hasta el dato, con el fin de armar el código para extraer la información. La estructura se ve de esta manera:

![webscraping](../imagenes/webscraping.png)

Se puede ver que la tabla de interés se encuentra dentro de una etiqueta `<table>`, pero hay mas de una de estas etiquetas en toda la página, por lo tanto se debe iterar por todas las etiquetas `<table>` que encuentre hasta encontrar un encabezado `<th>` que contenga los atributos `colspan='3'` y `class='navbox-title'`. Luego, se verifica si se encuentra el encabezado de la tabla buscada, en este caso "Población histórica". De encontrarse esto, toma la información de la tabla y la guarda en una variable `target_table`.

In [30]:
# Se inicializa una variable llamada target_table como None
target_table = None

# Se buscan todas las etiquetas de tabla (<table>) en el contenido HTML
tables = soup.find_all('table')

# Se itera a través de todas las tablas encontradas en la página web
for table in tables:
    # Se busca una etiqueta de encabezado (<th>) dentro de la tabla que tenga los atributos colspan='3' y class='navbox-title'
    header = table.find('th', {'colspan': '3', 'class': 'navbox-title'})
    
    # Se comprueba si se encontró un encabezado y si el texto del encabezado contiene "Población histórica"
    if header and "Población histórica" in header.text:
        # Si se cumple la condición anterior, asigna la tabla actual a la variable target_table y sale del bucle for con break
        target_table = table
        break

En la siguiente celda se observa la información extraída.

In [31]:
target_table

<table class="toccolours" style="width:15em;border-spacing: 0;float:right;clear:right;margin:0 0 1em 1em;"><tbody><tr><th class="navbox-title" colspan="3" style="padding:0.25em;font-size:110%">Población histórica</th></tr><tr style="font-size:95%"><th style="border-bottom:1px solid black;padding:1px;width:3em">Año</th><th style="border-bottom:1px solid black;padding:1px 2px;text-align:right"><abbr title="Población">Pob.</abbr></th><th style="border-bottom:1px solid black;padding:1px;text-align:right"><abbr title="Cambio porcentual">±%</abbr></th></tr><tr><th style="text-align:center;padding:1px">1779 </th><td style="text-align:right;padding:1px">24 205</td><td style="text-align:right;padding:1px">—    </td></tr><tr><th style="text-align:center;padding:1px">1810 </th><td style="text-align:right;padding:1px">44 800</td><td style="text-align:right;padding:1px">+85.1%</td></tr><tr><th style="text-align:center;padding:1px">1869 </th><td style="text-align:right;padding:1px">177 797</td><td s

A continuación, se extrae los datos de `target_table` y los almacena en un DataFrame de Pandas.

In [32]:
# Se comprueba si se encontró la tabla
if target_table:
    # Se crean listas para almacenar las columnas de la tabla
    years = []  
    populations = []  
    changes = [] 
    
    # Se inicializa una variable para almacenar el año actual
    current_year = None
    
    # Se itera a través de las filas de la tabla
    for row in target_table.find_all('tr'):
        # Se busca tanto en th (encabezados) como en td (celdas de datos)
        columns = row.find_all(['th', 'td'])  
        
        # Se comprueba si la fila tiene tres columnas (Año, Población y Cambio porcentual)
        if len(columns) == 3:
            current_year = columns[0].get_text(strip=True)  # Obtiene el año
            population = columns[1].get_text(strip=True)  # Obtiene la población
            change = columns[2].get_text(strip=True)  # Obtiene el cambio porcentual
            years.append(current_year)  # Agrega el año a la lista years
            populations.append(population)  # Agrega la población a la lista populations
            changes.append(change)  # Agrega el cambio porcentual a la lista changes
        elif len(columns) == 2 and current_year:
            # Si la fila tiene dos columnas y ya tenemos el año actual, 
            # se asume que es la población y el cambio porcentual
            population = columns[0].get_text(strip=True)  # Obtiene la población
            change = columns[1].get_text(strip=True)  # Obtiene el cambio porcentual
            populations.append(population)  # Agrega la población a la lista populations
            changes.append(change)  # Agrega el cambio porcentual a la lista changes
    
    # Se crea un DataFrame de Pandas con los datos extraídos
    df = pd.DataFrame({'Año': years, 'Población': populations, 'Cambio porcentual': changes})
    
    print("Ya se guardó el dataframe")
else:
    print("No se encontró la tabla deseada en la página.")


Ya se guardó el dataframe


Se observa el dataframe creado.

In [33]:
df

,Año,Población,Cambio porcentual
0,Año,Pob.,±%
1,1779,24 205,—
2,1810,44 800,+85.1%
3,1869,177 797,+296.9%
4,1895,663 854,+273.4%
5,1914,1 575 814,+137.4%
6,1947,2 981 043,+89.2%
7,1960,2 966 634,−0.5%
8,1970,2 972 453,+0.2%
9,1980,2 922 829,−1.7%


Este dataframe requiere de algunas limpiezas. Por un lado, la primer fila es en realidad del encabezado de la tabla, por lo que se debe eliminar. Por otro lado, la columna Cambio porcentual no se utilizará, por lo que también se eliminará. Finalmente, se eliminarán los espacios separación de miles en las cantidades de la Población.

In [34]:
# se elimina la primer fila
df = df.iloc[1:]
# Se elimina la columna Cambio porcentual
df = df.drop('Cambio porcentual', axis=1)
# Se sacan los espacios de separación de miles
df['Población'] = df['Población'].str.replace(' ', '').str.replace('\xa0', '').astype(int)
df

,Año,Población
1,1779,24205
2,1810,44800
3,1869,177797
4,1895,663854
5,1914,1575814
6,1947,2981043
7,1960,2966634
8,1970,2972453
9,1980,2922829
10,1991,2965403


Antes de finalizar, se verifican los tipos de datos de cada columna.

In [35]:
utils.verificar_tipo_variable(df)

,nombre_campo,tipo_datos
0,Año,[<class 'str'>]
1,Población,[<class 'int'>]


## Carga de los datos limpios

In [36]:
archivo = 'datos/poblacionCABA.csv'
df.to_csv(archivo, index=False, encoding='utf-8')
print(f'Se guardó el archivo {archivo}')

Se guardó el archivo datos/poblacionCABA.csv
